In [16]:
import tkinter as tk
from tkinter import messagebox
import serial

# Global variables for mode_label and window
uart = None
window = None
mode_label = None

# Initialize UART connection
def initialize_uart(port="COM6", baudrate=9600):  
    try:
        uart = serial.Serial(port, baudrate, timeout=1)
        print(f"Connected to {port} at {baudrate} baud.")
        return uart
    except serial.SerialException as e:
        print(f"Error connecting to UART: {e}")
        return None

def send_uart_command(command):
    if uart:
        if isinstance(command, str):  # Handle string commands
            uart.write(command.encode('utf-8'))  # Send string as bytes
            print(f"Sent: {command}")
        elif isinstance(command, list):  # Handle list of bytes (list of integers)
            uart.write(bytes(command))  # Send as bytes
            print(f"Sent: {command}")
        else:
            print("Invalid command. Must be a string or a list of integers.")
    else:
        print("UART not initialized.")

# Simulate receiving and processing data from UART
def uart_receive_and_process():
    if uart:
        try:
            # Read a line of data (terminated with '\n') from UART
            data = uart.readline().decode('utf-8').strip()
            if data:
                print(f"Received: {data}")  # Debug print the received string
                update_mode(f"{data}", "gray")  # Default for unrecognized commands
        except Exception as e:
            print(f"Error reading UART data: {e}")
    else:
        print("UART not initialized.")

# Update the mode display in the GUI
def update_mode(message, color):
    global mode_label
    global window

    mode_label.config(text=f"Mode: {message}")  # Update the label text
    window.config(bg=color)  # Change background color based on LED status

# Open the Diagnostic Page with 5 Buttons
def open_diagnostic_page():
    diagnostic_window = tk.Toplevel(window)
    diagnostic_window.title("Diagnostic Page")
    diagnostic_window.config(bg="#2c2f38")
    send_uart_command('1')

    # Button 1: Read DTC "Lost Communication"
    tk.Button(diagnostic_window, text="Read DTC: Lost Communication", command=lambda: send_uart_command('2'), bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 2: Read DTC "Over Heat"
    tk.Button(diagnostic_window, text="Read DTC: Over Heat", command=lambda: send_uart_command('3'), bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 3: Read DTC "Fault State"
    tk.Button(diagnostic_window, text="Read DTC: Fault State", command=lambda: send_uart_command('4'), bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 4: Clear DTC
    tk.Button(diagnostic_window, text="Clear DTC", command=lambda: [send_uart_command('6'), diagnostic_window.destroy()], bg="red", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 5: Exit DTC
    tk.Button(diagnostic_window, text="Exit DTC", command=lambda: [send_uart_command('5'), diagnostic_window.destroy()], bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)



# Open the Testing Page with Main Buttons
def open_testing_page():
    testing_window = tk.Toplevel(window)
    testing_window.title("Testing Page")
    testing_window.config(bg="#2c2f38")
    send_uart_command('8')

    # Button 1: Open GPIO Page
    tk.Button(testing_window, text="GPIO Controls", command=open_gpio_page, bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 2: Open ADC Page
    tk.Button(testing_window, text="ADC Readings", command=open_adc_page, bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 3: Exit Testing Page
    tk.Button(testing_window, text="Exit Testing", command=lambda: [send_uart_command('5'), testing_window.destroy()], bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

# Open GPIO Page with LED Buttons
def open_gpio_page():
    gpio_window = tk.Toplevel(window)
    gpio_window.title("GPIO Controls")
    gpio_window.config(bg="#2c2f38")
    send_uart_command('1')


    # Button 1: Test Red LED
    tk.Button(gpio_window, text="Test Red LED", command=lambda: send_uart_command('6'), bg="red", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 2: Test Blue LED
    tk.Button(gpio_window, text="Test Blue LED", command=lambda: send_uart_command('2'), bg="blue", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 3: Test Green LED
    tk.Button(gpio_window, text="Test Green LED", command=lambda: send_uart_command('3'), bg="green", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 4: Test White LED
    tk.Button(gpio_window, text="Test White LED", command=lambda: send_uart_command('4'), bg="white", fg="black", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 5: Exit GPIO Page
    tk.Button(gpio_window, text="Exit GPIO Controls", command=gpio_window.destroy, bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

# Open ADC Page with Sensor Buttons
def open_adc_page():
    adc_window = tk.Toplevel(window)
    adc_window.title("ADC Readings")
    adc_window.config(bg="#2c2f38")
    send_uart_command('2')

    # Button 1: Test Temperature Sensor
    tk.Button(adc_window, text="Test Temperature Sensor",command=lambda: send_uart_command('1'), bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)

    # Button 2: Exit ADC Page
    tk.Button(adc_window, text="Exit ADC Readings",command=adc_window.destroy, bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=10, fill="x", padx=20)


# Create the GUI
def create_gui():
    global mode_label
    global window

    window = tk.Tk()
    window.title("Tiva C UART Communication")
    window.geometry("500x500")
    window.config(bg="#2c2f38")  # Dark background color

    # Add a header for the app title
    header_frame = tk.Frame(window, bg="#2c2f38")
    header_frame.pack(fill="x", pady=20)

    tk.Label(header_frame, text="OBD Diagnostic & Testing Tool", font=("Arial", 18, "bold"), fg="white", bg="#2c2f38").pack()

    # Diagnostic Button
    tk.Button(window, text="Diagnostic", command=open_diagnostic_page,bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=20, fill="x", padx=50)

    # Testing Button
    tk.Button(window, text="Testing", command=open_testing_page,bg="#444c56", fg="white", font=("Arial", 14, "bold")).pack(pady=20, fill="x", padx=50)

    # Mode display
    mode_label = tk.Label(window, text="Mode: None", bg="gray", fg="black", font=("Arial", 16, "bold"))
    mode_label.pack(pady=20)

    # Status bar
    tk.Label(window, text="Ready", bd=1, relief="sunken", anchor="w",font=("Arial", 10), bg="#444c56", fg="white").pack(side="bottom", fill="x")

    # Periodically check for UART input
    def periodic_check():
        uart_receive_and_process()
        window.after(100, periodic_check)  # Check every 100ms

    periodic_check()
    window.mainloop()

# Main program
if __name__ == "__main__":
    # Initialize UART 
    uart = initialize_uart(port="COM6", baudrate=9600)  

    if uart:
        # Start the GUI
        create_gui()

        # Close UART connection when the GUI is closed
        uart.close()
        print("UART connection closed.")
    else:
        print("Failed to initialize UART.")

Connected to COM6 at 9600 baud.
Sent: 1
Received: ECU2: Error unidentified state received
Sent: 6
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :76
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :76
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :75
Received: Motor temperature is normal :105
Received: Motor temperature is normal :114
Received: Motor temperature is normal :127
Received: ECU2: system is in FAULT_STATE:2 VOLT
Sent: 1
Received: ECU2: Error unidentified state received
Sent: 4
Received: DTC FAULT STATE CON